After a lot of research and letting ideas go, I settled on the Loan Prediction Problem

https://www.kaggle.com/altruistdelhite04/loan-prediction-problem-dataset

My first choice was to attempt to predict the next big video game trend on youtube for content creators, but I was unable to find a dataset that I could use. If I could find this data, I would prefer to attempt that challenge (I couldn't figure out how to use the data from the Google YouTube 8M, and it looks like it's 31 GBs)

https://research.google.com/youtube8m/